In [21]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [22]:
# Prepare DataFrame
df = pd.read_csv("./Ethereum Historical Data.csv", thousands=",")
df = df.loc[:, ['Date', 'High', 'Low', 'Open', 'Vol', 'Price']]
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
df.set_index('Date', inplace=True)

In [23]:
# Define training data and testing data
# Train model with lines of data after the 200th
# Use the first 200 lines of data to examine the output of prediction
train_data = df.iloc[200:]
test_data = df.iloc[:200]

In [ ]:
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    Y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+5]]
        X.append(row)
        label = df_as_np[i+5]